In [1]:
###DVR functions
import sys
#print(sys.path)
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt


def pot_1d_well(x):
    V=0
    if(x>1):
        V+=100000   
    if(x<0):
        V+=100000
    return V

def pot_harm_osz(x,k,x0):
    return 0.5*k* (x-x0)**2

def pot_double_well(x, labda=2,g=1/50):
    return -(1/4) * labda**2 * x**2 +g*x**4 + labda**4 / (64 *g)

def kin_inf(grid,hbar=1,m=1):#(-inf,inf) intervall
    N_grid=len(grid)
    T = np.zeros((N_grid,N_grid)) 
    for k in range(N_grid):
        for l in range(N_grid):
            if(k==l):
                T[k,k]= np.pi**2 /3
            if(k!=l):
                T[k,l]= (-1)**(k-l) *2/((k-l)**2)
    T *= hbar**2 / (2*m* (grid[2]-grid[1])**2 )
    return T

def pot(grid, potential):#input: grid and generic potential
    N_grid=len(grid)
    V = np.zeros((N_grid,N_grid)) 
    for k in range(N_grid):
        V[k,k]= potential(grid[k])
    return V


In [2]:
###Checked: Harmonic Oscillator
if(False):
    for k in range(20):
        E=hbar *np.sqrt(spring/m)*(k+0.5)
        print(E,w[k],k, E-w[k])
###plot wavefkt
if(False):
    N=len(grid)
    HO_pot = np.zeros(N)
    for k in range(N):
        HO_pot[k]= pot_harm_osz(grid[k],k=spring,x0=0)
    for n in range(4):
        #plt.plot(grid,v[:,n]*10+w[n], label = "psi %i" % n)#*10 s.t visible (b.c. it is normalized anyways)
        plt.plot(grid,v[:,n]**2 *200+w[n], label = "psi_sq %i" % n)#*10 s.t visible (b.c. it is normalized anyways)
        plt.plot(grid,np.zeros_like(v[:,n])*10+w[n],'--')#*10 s.t visible (b.c. it is normalized anyways)
    plt.plot(grid,HO_pot,label= "V",linewidth=3 )
    plt.ylabel('E and Psi')
    plt.xlabel('x')
    #plt.ylim([0,w[9]])
    plt.xlim([-7,7])
    plt.legend()
    plt.show()


In [3]:
###Checking: Double Well....#strongly dependent on given conditions, plotting is adviced
if(False):
    hbar=1 #like in OTOC paper
    m=1/2 #like in OTOC paper
    grid = np.linspace(-7,7,2000,endpoint=False)[1:] #in principle: gridspacing one to 5 grid points per de broglie wavelength and where one expects wavefkt to still be
    H=pot(grid=grid, potential=lambda x : pot_double_well(x=x,labda= 2, g=1/50))+kin_inf(grid=grid,hbar=hbar,m=m)
    w,v = LA.eigh(H)

    for k in range(10):
        print(w[k],k)
    for k in range(10):
        print(w[k+1]-w[k],k+1,'->',k)
###plot wavefkt
if(False):
    N=len(grid)
    DW_pot = np.zeros(N)
    for k in range(N):
        DW_pot[k]= pot_double_well(grid[k],labda=2, g=1/50)
    for n in range(13):
        plt.plot(grid,v[:,n]*20+w[n], label = "psi %i" % n)#*10 s.t visible (b.c. it is normalized anyways)
        #plt.plot(grid,v[:,n]**2 *1600+w[n], label = "psi_sq %i" % n)#*10 s.t visible (b.c. it is normalized anyways)
        plt.plot(grid,np.zeros_like(v[:,n])+w[n],'--')#*10 s.t visible (b.c. it is normalized anyways)
    plt.plot(grid,DW_pot,label= "V",linewidth=3 )
    plt.ylabel('E and Psi')
    plt.xlabel('x')
    #plt.ylim([0,w[20]])
    plt.xlim([-8,8])
    #plt.legend()
    plt.show()


In [4]:
###checked for Particle in a box
if(False):
    for k in range(5):
        print(E_n(k+1),w[k],k)#paricle in a box
###plot wavefkt
if(False):
    n=4
    plt.plot(grid,v[:,n], label = "psi")
    psi = np.zeros(len(grid))
    psi_sq = np.zeros_like(psi)
    for k in range(len(grid)):
        psi[k]=psi_n_bound(grid[k],n)
        psi_sq[k]=psi[k]**2
    #plt.plot(grid,psi, label = "psi")
    #plt.plot(grid,psi_sq, label = "psi **2")
    plt.ylabel('psi')
    plt.xlabel('x')
    #plt.legend()
    plt.show()


In [5]:
###DVR 
#parameters
hbar=1 #like in OTOC paper
m=1/2 #like in OTOC paper
L=1
spring=1


#H=pot(grid=grid, potential=pot_1d_well)+kin_inf(grid=grid,hbar=hbar,m=m)
#H=pot(grid=grid, potential=lambda x : pot_harm_osz(x=x,k=spring,x0=0))+kin_inf(grid=grid,hbar=hbar,m=m)
###Double Well
grid = np.float32(np.linspace(-7,7,2000,endpoint=False)[1:]) #in principle: gridspacing one to 5 grid points per de broglie wavelength and where one expects wavefkt to still be
H=pot(grid=grid, potential=lambda x : pot_double_well(x=x,labda= 2, g=1/50))+kin_inf(grid=grid,hbar=hbar,m=m)

w,v = LA.eigh(H) #again look at scipy.space.linalg.eigsh at some point 

In [6]:
def E_n_DVR(n,w=w):
    return w[n]

def E_nm_DVR(n,m, w=w):
    return E_n_DVR(n,w=w)-E_n_DVR(m,w=w)

def x_nm_from_DVR(v,n,m):
    return np.sum(v[:,n]*v[:,m]*grid)

def b_nm(t,n,m,X,E_nm_fk=E_nm_DVR): 
    I=0
    for k in range(len(X[:,1])):
        I+=X[n,k]*X[k,m]*(E_nm_fk(k,m)*np.exp(E_nm_fk(n,k)*1j*t)-E_nm_fk(n,k)*np.exp(E_nm_fk(k,m)*1j*t))
    return 0.5*I

def c_n(t,n,X,E_nm_fk=E_nm_DVR): #microcanonical OTOC
    I=0
    for m in range(len(X[:,1])):
        I += np.abs(b_nm(t,n,m,X,E_nm_fk))**2
        #if(n==13 or n==14):
        #    print(np.abs(b_nm(t,n,m,X,E_nm_fk))**2,n)
    return I


def C_T(C,beta=1,E_n_fk=E_n_DVR): #C is microcanonical OTOC
    Z=0 
    sum=np.zeros_like(C[:,1])#get Thermal OTOC for each time t
    for n in range(len(C[1,:])):
        Z+=np.exp(-beta *E_n_fk(n)) #partition sum
    for n in range(len(C[1,:])):
        for time in range(len(C[:,1])):
            sum[time] += np.exp(-beta *E_n_fk(n))*C[time,n] #wights of all microcanonical OTOCS
    return sum/Z

In [7]:
x_range=150
X=np.zeros((x_range,x_range))
for n in range(x_range):
    for m in range(x_range):
        ###DVR
        X[n,m]=x_nm_from_DVR(v=v,n=n,m=m)

In [8]:
from mylib import OTOC1D

X_Fortran= np.zeros((x_range,x_range), np.float64)
N_trunc=150
EV=np.float64(v[:,0:N_trunc])
N_grdpts=len(grid)
OTOC1D.otoc_1d.which_version()



 Version:1


In [9]:
#Y=OTOC1D.otoc_1d.fill_x_nm(X_Fortran,EV, grid, N_grdpts,N_trunc)
print(np.shape(X_Fortran))#f32
print(np.shape(EV))#f32
#print(np.finfo(grid[1]))#f32
#print(np.iinfo(N_trunc))#i64
#print(np.iinfo(N_grdpts))#i64


(150, 150)
(1999, 150)


In [1]:
import sys
import numpy as np
#print(sys.path)
from mylib import OTOC1D
N_trunc2 = 50#6,150, didn't work starting 62
N_grdpts2 = 1999#10,1999
grid2 = np.linspace(1,N_grdpts2,N_grdpts2)
EV2 = np.ones((N_grdpts2,N_trunc2))
X2=np.zeros((N_trunc2,N_trunc2))

In [2]:
#np.jadn
y=OTOC1D.otoc_1d.fill_x_nm(X2,EV2,grid2, N_grdpts2,N_trunc2)

#print(np.finfo(X2[1,1]))#float64
#print(np.finfo(EV2[1,1]))#f64
#print(np.finfo(grid2[1]))#f64
#print(np.iinfo(N_trunc2))#i64
#print(np.iinfo(N_grdpts2))#i64


In [ ]:
np.srfge

In [ ]:
#t=np.linspace(0,0.7,300) #good range for PiB
#t=np.linspace(0,20,3000)# HO
t=np.linspace(0,4,600)# DW
C = np.zeros((len(t),x_range))
for k in range(len(t)):
    for n in range(x_range):
        C[k,n] = c_n(t[k],n, X,E_nm_fk=E_nm_DVR) 
        #if(k==1):
        #  print(c_n(t[k],n, X,E_nm_fk=E_nm_DVR),n)
C=C[:,:-1] #because energy difference is wrong for the last one, I think because we don't have the last energy!

    1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00       1999000.00

KeyboardInterrupt: 

In [ ]:
if(True):###plot microcanonical OTOC
    import matplotlib.pyplot as plt
    from matplotlib.ticker import ScalarFormatter
    fig, ax = plt.subplots()
    ax.plot(t,np.zeros_like(C[:,2]), '--')
    for n in (1,5,9,11,13,20):#DW OTOC from Paper (in paper:20)
        ax.plot(t,np.log(C[:,n]), label = "n = %i" %n)
    ax.set_ylabel('OTOC')
    #ax.set_yscale('log') #PiB #DW
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.set_xlabel('t')
    ax.set_xlim([0,4])# [0,4]..DW OTOC Paper
    ax.legend()
    plt.show()


In [ ]:
if(True):#Plot Thermal OTOC
    import matplotlib.pyplot as plt
    from matplotlib.ticker import ScalarFormatter
    fig, ax = plt.subplots()
    ax.plot(t,np.zeros_like(C[:,2]), '--')#depends if log or not
    for T in (1,5,9,30):#DW
    #for T in (1,10,40,100): #HO, doesn't matter since all the same
    #for T in (1,20,50,100,200):#PiB
        beta=1/T
        #ax.plot(t,C_T(C,beta=beta,E_n_fk=E_n_DVR), label = "T = %.2f" %T)
        ax.plot(t,np.log(C_T(C,beta=beta,E_n_fk=E_n_DVR)), label = "T = %.2f" %T)
    ax.set_ylabel('OTOC')
    #ax.set_yscale('log')
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.set_xlabel('t')
    ax.set_xlim([0,4])
    #plt.text( x= 5,y=0.08, s=r'$\beta$')
    ax.legend()
    plt.show()
